### Jupyter Notebook 단축키
- ctrl+enter: 셀 실행
- shift+enter: 셀 실행 및 다음 셀 이동
- alt+enter: 셀 실행, 다음 셀 이동, 새로운 셀 생성
- a: 상단에 새로운 셀 만들기
- b: 하단에 새로운 셀 만들기
- dd: 셀 삭제(x: 셀 삭제)
- esc: 셀 나가기
- m: 마크다운 셀로 바꾸기
- y: 마크다운 셀을 다시 Code로 바꾸기

# 08장_사전 훈련된 모델 다루기

* 개방적인 환경에서 빠르게 학습하기 위함
* 전이 학습 (transfer learning), 신경 스타일 전이 (neural sytle transfer)

### 8.1 텐서플로 허브
* TensorFlow Hub는 재사용 가능한 모델을 쉽게 이용할 수 있는 라이브러리 

In [1]:
import tensorflow as tf

import numpy as np

In [10]:
# !pip install --upgrade tensorflow-hub

In [2]:
# 텐서플로 허브에서 사전 훈련된 MobileNet 모델 불러오기

import tensorflow_hub as hub

mobile_net_url = "https://tfhub.dev/google/tf2-preview/mobilenet_v2/classification/2"
model = tf.keras.Sequential([
    hub.KerasLayer(handle=mobile_net_url, input_shape=(224, 224, 3), trainable=False)
])
model.summary()

# MobileNet은 계산 부담이 큰 conv. 신경망을 연산 성능이 제한된 모바일 환경에서도 작동 가능하도록 네트워크 구조를 경량화 한 것
# hub.KerasLayer()를 통해 tf.keras에서 사용 가능한 레이어로 변환


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer (KerasLayer)     (None, 1001)              3540265   
Total params: 3,540,265
Trainable params: 0
Non-trainable params: 3,540,265
_________________________________________________________________


In [5]:
# ImageNetV2-TopImages 불러오기
# 각 클래스에서 가장 많은 선택을 받은 이미지 10장씩을 모아놓은 10,000장의 이미지가 포함된 topimages 데이터셋 사용

import os
import pathlib

content_data_url = '/content/sample_data'
data_root_orig = tf.keras.utils.get_file('imagenetV2', 'https://s3-us-west-2.amazonaws.com/imagenetv2public/imagenetv2-top-images.tar.gz', cache_dir=content_data_url, extract=True)
data_root = pathlib.Path(content_data_url + "/datasets/imagenetv2-topimages")
print(data_root)

\content\sample_data\datasets\imagenetv2-topimages


In [13]:
# !pip install nltk

In [14]:
# ImageNet 라벨 텍스트 불러오기

label_file = tf.keras.utils.get_file('label', 'https://storage.googleapis.com/download.tensorflow.org/data/ImageNetLabels.txt')
label_text = None
with open(label_file, 'r') as f:
    label_text = f.read().split('\n')[:-1]
print(len(label_text))
print(label_text[:10])
print(label_text[-10:])

# 폴더 이름이 wordnet의 단어로 수정되었기 때문에, nltk 패키지에서 wordnet을 다운받기
import nltk
nltk.download('wordnet')
from nltk.corpus import wordnet

# wordnet과 인터넷에 올라온 label 텍스트는 조금씩 다르기 때문에 차이를 없애기 위해서 아래의 전처리 작업을 진행합니다.
label_text = [c.lower().replace('-','').replace('_','').replace(' ','') for c in label_text]

1001
['background', 'tench', 'goldfish', 'great white shark', 'tiger shark', 'hammerhead', 'electric ray', 'stingray', 'cock', 'hen']
['buckeye', 'coral fungus', 'agaric', 'gyromitra', 'stinkhorn', 'earthstar', 'hen-of-the-woods', 'bolete', 'ear', 'toilet tissue']


[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\seunghwan\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\wordnet.zip.


In [16]:
# 이미지 확인 

import PIL.Image as Image
import matplotlib.pyplot as plt
import random

all_image_paths = list(data_root.glob('*/*'))
all_image_paths = [str(path) for path in all_image_paths]
# 이미지를 랜덤하게 섞습니다.
random.shuffle(all_image_paths)

image_count = len(all_image_paths)
print('image_count:', image_count)

# plt.figure(figsize=(12,12))
# for c in range(9):
#     image_path = random.choice(all_image_paths)
#     plt.subplot(3,3,c+1)
#     plt.imshow(plt.imread(image_path))
#     # idx = int(image_path.split('/')[-2]) + 1
#     # plt.title(str(idx) + ', ' + label_text[idx])
#     word = wordnet.synset_from_pos_and_offset('n',int(image_path.split('/')[-2][1:]))
#     word = word.name().split('.')[0].replace('-','').replace('_','').replace(' ','')
#     plt.title(str(label_text.index(word)) + ', ' + word)
#     plt.axis('off')
# plt.show()

image_count: 0


In [18]:
data_root

WindowsPath('/content/sample_data/datasets/imagenetv2-topimages')